In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
bhoj_train =  pd.read_csv('/content/bmmnerdataset/Bhojpuri_ner.remove[1].train', sep="\t",header=None)


In [ ]:
bhoj_train.head(100)

,0,1
0,दोसर,OTHER
1,ई,OTHER
2,कि,OTHER
3,ई,OTHER
4,पूरा,OTHER
...,...,...
95,जुडि़हें,OTHER
96,",",OTHER
97,तब,OTHER
98,ले,OTHER


**Model using only bhojpuri language(45 labels(ALL))-IndicNER and IndicBertV2**

In [ ]:
raw_bh_tr = open('/content/drive/MyDrive/bmmnerdataset/Bhojpuri_ner.remove[1].train').readlines()

In [ ]:
def preprocess(raw):
  all_x = []

  point = []

  for line in raw:
      stripped_line = line.strip().split('\t')
      point.append(stripped_line)
      if line == '\n':
          all_x.append(point[:-1])
          point = []

  all_x = all_x[:-1]
  X = [[c[0] for c in x] for x in all_x]
  Y = [[c[1] for c in y] for y in all_x]
  return X,Y


In [ ]:
bh_X_tr,bh_Y_tr=preprocess(raw_bh_tr)

In [ ]:
print(bh_X_tr[2])
print(bh_Y_tr[2])

['माई', 'बाप', 'बिहीन', 'टूअर', 'लइकी', 'के', 'बिकल', 'आर्त', 'भाव', 'पर', 'कुन्ती', 'के', 'मन', 'भरि', 'आइल', '।']
['OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'B-Person', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER']


In [ ]:
raw_mt_tr = open('/content/drive/MyDrive/bmmnerdataset/Maithili_ner[1].train').readlines()
mt_X_tr,mt_Y_tr=preprocess(raw_mt_tr)

In [ ]:
all_labels = [c for y in mt_Y_tr for c in y]

n_labels = len(set(all_labels))

In [ ]:
labels=list(set(all_labels))
labels.sort()

In [ ]:
set(all_labels)

{'B-Artifact',
 'B-Count',
 'B-Date',
 'B-Day',
 'B-Disease',
 'B-Distance',
 'B-Entertainment',
 'B-Facility',
 'B-Location',
 'B-Locomotive',
 'B-Material',
 'B-Money',
 'B-Month',
 'B-Organism',
 'B-Organization',
 'B-Period',
 'B-Person',
 'B-Plant',
 'B-Quantity',
 'B-Special_Day',
 'B-Time',
 'B-Year',
 'I-Artifact',
 'I-Count',
 'I-Date',
 'I-Day',
 'I-Disease',
 'I-Distance',
 'I-Entertainment',
 'I-Facility',
 'I-Location',
 'I-Locomotive',
 'I-Material',
 'I-Money',
 'I-Month',
 'I-Organism',
 'I-Organization',
 'I-Period',
 'I-Person',
 'I-Plant',
 'I-Quantity',
 'I-Special_Day',
 'I-Time',
 'I-Year',
 'OTHER'}

In [ ]:
label2ind = {label: (index) for index, label in enumerate(labels)}

In [ ]:
def cat(y):
  return [[label2ind[w] for w in s] for s in y]

In [ ]:
label_list = labels

In [ ]:
bh_Y_tr = cat(bh_Y_tr)

In [ ]:
bh_Y_tr[2]

[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 16, 44, 44, 44, 44, 44]

In [ ]:
len(bh_X_tr)

11543

In [ ]:
df_bh=pd.DataFrame(list(zip(bh_X_tr,bh_Y_tr)),columns=['tokens','ner_tags'])

In [ ]:
df_bh.head(20)

,tokens,ner_tags
0,"[दोसर, ई, कि, ई, पूरा, कहानी, कल्पना, ह, ,, कव...","[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 4..."
1,"[जाने, केतना, मकानन, के, रूख, मरा, गइल, रहे, ।]","[44, 44, 44, 44, 44, 44, 44, 44, 44]"
2,"[माई, बाप, बिहीन, टूअर, लइकी, के, बिकल, आर्त, ...","[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 16, 4..."
3,"[सिकुमार, ओही, गलती, में, पकड़ाइल, रहलें, ।]","[16, 44, 44, 44, 44, 44, 44]"
4,"[जब, ले, भोजपुरी, कहानी, अपना, समय, के, नबुज, ...","[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 4..."
5,"[एकरा, बाद, सुकन्या, के, अपना, बाप, ,, राजा, स...","[44, 44, 16, 44, 44, 44, 44, 44, 16, 44, 44, 4..."
6,"[आ, जब, रुसी, सैनिक, आवऽतारे, स, आ, ऊ, धधा, के...","[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 4..."
7,[।],[44]
8,"[समाज, जवाबदेही, तय, कऽ, के, अइसन, काम, करा, स...","[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 4..."
9,"[4, ,, 9, ,, 14, तारीख, अउरी, मंगलवार, के, एह,...","[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 4..."


In [ ]:
from datasets import Dataset

In [ ]:
dataset_bh_tr=Dataset.from_pandas(df_bh)

In [ ]:
dataset_bh_tr

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 11543
})

In [ ]:
dataset_bh_ev=dataset_bh_tr.train_test_split(test_size=0.15)

In [ ]:
dataset_bh_ev

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 9811
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1732
    })
})

In [ ]:
raw_bh_te = open('/content/drive/MyDrive/bmmnerdataset/Bhojpuri_ner.remove[1].test').readlines()

In [ ]:
bh_X_te,bh_Y_te=preprocess(raw_bh_te)

In [ ]:
bh_Y_te = cat(bh_Y_te)

In [ ]:
df_bh_te=pd.DataFrame(list(zip(bh_X_te,bh_Y_te)),columns=['tokens','ner_tags'])

In [ ]:
dataset_bh_te=Dataset.from_pandas(df_bh_te)

In [ ]:
# Load model directly
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER",num_labels=n_labels,ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai4bharat/IndicNER and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([45, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([45]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBERTv2-MLM-only")
#model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicBERTv2-MLM-only",num_labels=n_labels)

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.75M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
v=tokenizer.get_vocab()

In [ ]:
from transformers import AutoModelForMaskedLM,  pipeline

In [ ]:
model = AutoModelForMaskedLM.from_pretrained("ai4bharat/IndicBERTv2-MLM-only")

config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/IndicBERTv2-MLM-only were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [ ]:
sentence = '[MASK] अर्थशास्त्र के रचयिता चाणक्य के बारे में कुछ कहला से चुपे रहल ज्यादा श्रद्धास्पद होई ।' #bhojpuri

In [ ]:
sentence='[MASK] नेताजीक संग फिल्मी कलाकार सेहो आबि रहल छथिन्ह ।' #maithili

In [ ]:
sentence='[MASK] बेसिसँ-बेसि सभा , रैली कऽ लेबय चाहय छथिन्ह ।'

In [ ]:
pred = pipe(sentence)

In [ ]:
pred

[{'score': 0.9875955581665039,
  'token': 16,
  'token_str': '<mai>',
  'sequence': 'बेस ##िसँ - बेस ##ि सभा, रैली कऽ ले ##बय चाह ##य छथि ##न्ह ।'},
 {'score': 0.005187340546399355,
  'token': 12,
  'token_str': '<hi>',
  'sequence': 'बेस ##िसँ - बेस ##ि सभा, रैली कऽ ले ##बय चाह ##य छथि ##न्ह ।'},
 {'score': 0.0008205327321775258,
  'token': 20,
  'token_str': '<ne>',
  'sequence': 'बेस ##िसँ - बेस ##ि सभा, रैली कऽ ले ##बय चाह ##य छथि ##न्ह ।'},
 {'score': 0.0008178717689588666,
  'token': 1314,
  'token_str': 'ओ',
  'sequence': 'ओ बेस ##िसँ - बेस ##ि सभा, रैली कऽ ले ##बय चाह ##य छथि ##न्ह ।'},
 {'score': 0.0002721907221712172,
  'token': 17708,
  'token_str': 'लोक',
  'sequence': 'लोक बेस ##िसँ - बेस ##ि सभा, रैली कऽ ले ##बय चाह ##य छथि ##न्ह ।'}]

In [ ]:
'[MASK]' in v

True

In [ ]:
model=model.to("cuda")

In [ ]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
train_dataset = dataset_bh_ev["train"]
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset (num_proc=4):   0%|          | 0/9811 [00:00<?, ? examples/s]

In [ ]:
eval_dataset = dataset_bh_ev["test"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on eval dataset",
)

Running tokenizer on eval dataset (num_proc=4):   0%|          | 0/1732 [00:00<?, ? examples/s]

In [ ]:
test_dataset = dataset_bh_te
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset (num_proc=4):   0%|          | 0/4947 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from datasets import load_metric

In [ ]:
!pip3 install sentencepiece
!pip3 install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 452.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=33bb956c5ca0bfa19ed508cfc77b5d03cc3c08f7830e6fac6f39569cb92a9f59
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# Metrics
metric = load_metric("seqeval")

<ipython-input-16-c636c133c557>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
def compute_metrics(p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)

        # Remove ignored index (special tokens)
        true_predictions = [
            [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]

        results = metric.compute(predictions=true_predictions, references=true_labels)
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.3 MB/s eta 0:00:00


In [ ]:
pip install accelerate -U         #restart session after these 2 installs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    load_best_model_at_end = True,
    evaluation_strategy = IntervalStrategy.STEPS,
    eval_steps = 500,
    metric_for_best_model = "eval_f1")

In [ ]:
# Initialize our Trainer
#early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
#args.metric_for_best_model = "f1"
#args.load_best_model_at_end = True
#args.evaluation_strategy = IntervalStrategy.STEPS
#args.eval_steps = args.save_steps
#args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #callbacks=[early_stopping_callback],
    args=args,
)

In [ ]:
train_result = trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.327000,0.160235,0.762162,0.408696,0.532075,0.965785
1000,0.143600,0.124835,0.718050,0.526570,0.607581,0.970588
1500,0.106600,0.108751,0.751530,0.593237,0.663067,0.971937
2000,0.089300,0.101263,0.681018,0.672464,0.676714,0.972105
2500,0.083400,0.097564,0.711340,0.666667,0.688279,0.974043
3000,0.061700,0.098393,0.728665,0.643478,0.683427,0.974381
3500,0.060900,0.096422,0.695866,0.683092,0.689420,0.973411


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packa

In [ ]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.9734
  eval_f1                 =     0.6894
  eval_loss               =     0.0964
  eval_precision          =     0.6959
  eval_recall             =     0.6831
  eval_runtime            = 0:00:59.53
  eval_samples_per_second =     29.094
  eval_steps_per_second   =      3.645


In [ ]:
preds,labels,metrics2=trainer.predict(test_dataset,metric_key_prefix="predict")

In [ ]:
metrics2  #IndicBertV2

{'predict_loss': 0.09061162918806076,
 'predict_precision': 0.671858022455632,
 'predict_recall': 0.660377358490566,
 'predict_f1': 0.6660682226211848,
 'predict_accuracy': 0.9722919326670483,
 'predict_runtime': 164.8993,
 'predict_samples_per_second': 30.0,
 'predict_steps_per_second': 3.754}

In [ ]:
metrics2  #IndicBertV2 (Per Entity)

{'predict_loss': 0.09061162918806076,
 'predict_Artifact_precision': 0.5051546391752577,
 'predict_Artifact_recall': 0.5764705882352941,
 'predict_Artifact_f1': 0.5384615384615384,
 'predict_Artifact_number': 170,
 'predict_Count_precision': 0.375886524822695,
 'predict_Count_recall': 0.3333333333333333,
 'predict_Count_f1': 0.35333333333333333,
 'predict_Count_number': 159,
 'predict_Date_precision': 0.36,
 'predict_Date_recall': 0.5294117647058824,
 'predict_Date_f1': 0.42857142857142855,
 'predict_Date_number': 17,
 'predict_Day_precision': 0.3333333333333333,
 'predict_Day_recall': 0.5555555555555556,
 'predict_Day_f1': 0.4166666666666667,
 'predict_Day_number': 9,
 'predict_Disease_precision': 1.0,
 'predict_Disease_recall': 0.1111111111111111,
 'predict_Disease_f1': 0.19999999999999998,
 'predict_Disease_number': 9,
 'predict_Distance_precision': 0.0,
 'predict_Distance_recall': 0.0,
 'predict_Distance_f1': 0.0,
 'predict_Distance_number': 3,
 'predict_Entertainment_precision': 0

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/bh_indicbertv2')
model.save_pretrained('/content/drive/MyDrive/bh_indicbertv2')

In [ ]:
metrics2  #IndicNER

{'predict_loss': 0.14093327522277832,
 'predict_Artifact_precision': 0.45871559633027525,
 'predict_Artifact_recall': 0.29411764705882354,
 'predict_Artifact_f1': 0.3584229390681004,
 'predict_Artifact_number': 170,
 'predict_Count_precision': 0.34394904458598724,
 'predict_Count_recall': 0.33962264150943394,
 'predict_Count_f1': 0.34177215189873417,
 'predict_Count_number': 159,
 'predict_Date_precision': 0.2222222222222222,
 'predict_Date_recall': 0.35294117647058826,
 'predict_Date_f1': 0.27272727272727276,
 'predict_Date_number': 17,
 'predict_Day_precision': 0.0,
 'predict_Day_recall': 0.0,
 'predict_Day_f1': 0.0,
 'predict_Day_number': 9,
 'predict_Disease_precision': 0.0,
 'predict_Disease_recall': 0.0,
 'predict_Disease_f1': 0.0,
 'predict_Disease_number': 9,
 'predict_Distance_precision': 0.0,
 'predict_Distance_recall': 0.0,
 'predict_Distance_f1': 0.0,
 'predict_Distance_number': 3,
 'predict_Entertainment_precision': 0.5,
 'predict_Entertainment_recall': 0.10810810810810811

In [ ]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [list(set(all_labels))[t] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [ ]:
model.to('cpu')

In [ ]:
sentence = 'अर्थशास्त्र के रचयिता चाणक्य के बारे में कुछ कहला से चुपे रहल ज्यादा श्रद्धास्पद होई ।'

predicted_labels = get_predictions(sentence=sentence,
                                   tokenizer=tokenizer,
                                   model=model
                                   )

for index in range(len(sentence.split(' '))):
  print( sentence.split(' ')[index] + '\t' + predicted_labels[index] )

अर्थशास्त्र	OTHER
के	OTHER
रचयिता	OTHER
चाणक्य	B-Person
के	OTHER
बारे	OTHER
में	OTHER
कुछ	OTHER
कहला	OTHER
से	OTHER
चुपे	OTHER
रहल	OTHER
ज्यादा	OTHER
श्रद्धास्पद	OTHER
होई	OTHER
।	OTHER


In [ ]:
bh_X_te[2]

['‘अर्थशास्त्र’',
 'के',
 'रचयिता',
 'चाणक्य',
 'के',
 'बारे',
 'में',
 'कुछ',
 'कहला',
 'से',
 'चुपे',
 'रहल',
 'ज्यादा',
 'श्रद्धास्पद',
 'होई',
 '।']

**Combined model using all 3 languages(All 45 Labels)-IndicBertV2-SS**

In [ ]:
raw_mt_tr = open('/content/drive/MyDrive/bmmnerdataset/Maithili_ner[1].train').readlines()
mt_X_tr,mt_Y_tr=preprocess(raw_mt_tr)


In [ ]:
all_labels = [c for y in mt_Y_tr for c in y]

n_labels = len(set(all_labels))

In [ ]:
n_labels

45

In [ ]:
set(all_labels)

{'B-Artifact',
 'B-Count',
 'B-Date',
 'B-Day',
 'B-Disease',
 'B-Distance',
 'B-Entertainment',
 'B-Facility',
 'B-Location',
 'B-Locomotive',
 'B-Material',
 'B-Money',
 'B-Month',
 'B-Organism',
 'B-Organization',
 'B-Period',
 'B-Person',
 'B-Plant',
 'B-Quantity',
 'B-Special_Day',
 'B-Time',
 'B-Year',
 'I-Artifact',
 'I-Count',
 'I-Date',
 'I-Day',
 'I-Disease',
 'I-Distance',
 'I-Entertainment',
 'I-Facility',
 'I-Location',
 'I-Locomotive',
 'I-Material',
 'I-Money',
 'I-Month',
 'I-Organism',
 'I-Organization',
 'I-Period',
 'I-Person',
 'I-Plant',
 'I-Quantity',
 'I-Special_Day',
 'I-Time',
 'I-Year',
 'OTHER'}

In [ ]:
labels=list(set(all_labels))
labels.sort()
label2ind = {label: (index) for index, label in enumerate(labels)}

In [ ]:
def cat(y):
  return [[label2ind[w] for w in s] for s in y]

In [ ]:
for m in mt_X_tr:
  m.insert(0,'<mai>')

In [ ]:
for m in mt_Y_tr:
  m.insert(0,'OTHER')

In [ ]:
mt_X_tr[0]

['<mai>',
 'नेताजीक',
 'संग',
 'फिल्मी',
 'कलाकार',
 'सेहो',
 'आबि',
 'रहल',
 'छथिन्ह',
 '।']

In [ ]:
a=tokenizer.get_vocab()
print(tokenizer.all_special_tokens_extended)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [ ]:
mt_Y_tr = cat(mt_Y_tr)
df_mt_tr=pd.DataFrame(list(zip(mt_X_tr,mt_Y_tr)),columns=['tokens','ner_tags'])
dataset_mt_tr=Dataset.from_pandas(df_mt_tr)
dataset_mt_ev=dataset_mt_tr.train_test_split(test_size=0.15,seed=0)

In [ ]:
raw_mt_te = open('/content/drive/MyDrive/bmmnerdataset/Maithili_ner[1].test').readlines()
mt_X_te,mt_Y_te=preprocess(raw_mt_te)
mt_Y_te = cat(mt_Y_te)
df_mt_te=pd.DataFrame(list(zip(mt_X_te,mt_Y_te)),columns=['tokens','ner_tags'])
dataset_mt_te=Dataset.from_pandas(df_mt_te)

In [ ]:
raw_bh_tr = open('/content/drive/MyDrive/bmmnerdataset/Bhojpuri_ner.remove[1].train').readlines()
bh_X_tr,bh_Y_tr=preprocess(raw_bh_tr)
bh_Y_tr = cat(bh_Y_tr)
df_bh_tr=pd.DataFrame(list(zip(bh_X_tr,bh_Y_tr)),columns=['tokens','ner_tags'])
dataset_bh_tr=Dataset.from_pandas(df_bh_tr)
dataset_bh_ev=dataset_bh_tr.train_test_split(test_size=0.15,seed=0)

In [ ]:
raw_bh_te = open('/content/drive/MyDrive/bmmnerdataset/Bhojpuri_ner.remove[1].test').readlines()
bh_X_te,bh_Y_te=preprocess(raw_bh_te)
bh_Y_te = cat(bh_Y_te)
df_bh_te=pd.DataFrame(list(zip(bh_X_te,bh_Y_te)),columns=['tokens','ner_tags'])
dataset_bh_te=Dataset.from_pandas(df_bh_te)

In [ ]:
raw_mg_tr = open('/content/drive/MyDrive/bmmnerdataset/Magahi_ner[1].train').readlines()
mg_X_tr,mg_Y_tr=preprocess(raw_mg_tr)

In [ ]:
mg_Y_tr = cat(mg_Y_tr)
df_mg_tr=pd.DataFrame(list(zip(mg_X_tr,mg_Y_tr)),columns=['tokens','ner_tags'])
dataset_mg_tr=Dataset.from_pandas(df_mg_tr)
dataset_mg_ev=dataset_mg_tr.train_test_split(test_size=0.15,seed=0)

In [ ]:
raw_mg_te = open('/content/drive/MyDrive/bmmnerdataset/Magahi_ner[1].test').readlines()
mg_X_te,mg_Y_te=preprocess(raw_mg_te)


In [ ]:
mg_Y_te = cat(mg_Y_te)
df_mg_te=pd.DataFrame(list(zip(mg_X_te,mg_Y_te)),columns=['tokens','ner_tags'])
dataset_mg_te=Dataset.from_pandas(df_mg_te)

In [ ]:
# Load model directly
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBERTv2-SS")
model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicBERTv2-SS",num_labels=n_labels)

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-SS and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model=model.to("cuda")

In [ ]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=128,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
label_list=labels

In [ ]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

<ipython-input-26-435f98c90f62>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
pre_concatenated_train_split = []
pre_concatenated_train_split.append( dataset_bh_ev['train'] )
pre_concatenated_train_split.append( dataset_mt_ev['train'] )
pre_concatenated_train_split.append( dataset_mg_ev['train'] )

In [ ]:
pre_concatenated_validation_split = []
pre_concatenated_validation_split.append( dataset_bh_ev['test'] )
pre_concatenated_validation_split.append( dataset_mt_ev['test'] )
pre_concatenated_validation_split.append( dataset_mg_ev['test'] )

In [ ]:
pre_concatenated_test_split = []
pre_concatenated_test_split.append( dataset_bh_te )
pre_concatenated_test_split.append( dataset_mt_te )
pre_concatenated_test_split.append( dataset_mg_te )

In [ ]:
from datasets import concatenate_datasets, DatasetDict

concatenated_dataset = DatasetDict()
concatenated_dataset["train"] = concatenate_datasets(
    pre_concatenated_train_split
)

concatenated_dataset["validation"] = concatenate_datasets(
    pre_concatenated_validation_split
)
concatenated_dataset["test"] = concatenate_datasets(
    pre_concatenated_test_split
)


In [ ]:
train_dataset = concatenated_dataset["train"]
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset (num_proc=4):   0%|          | 0/20096 [00:00<?, ? examples/s]

In [ ]:
eval_dataset = concatenated_dataset["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on eval dataset",
)

Running tokenizer on eval dataset (num_proc=4):   0%|          | 0/3549 [00:00<?, ? examples/s]

In [ ]:
test_dataset = concatenated_dataset["test"]
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset (num_proc=4):   0%|          | 0/7976 [00:00<?, ? examples/s]

**IndicBertV2 for Maithili only**

In [ ]:
train_dataset = dataset_mt_ev["train"]
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

In [ ]:
eval_dataset = dataset_mt_ev["test"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on eval dataset",
)

In [ ]:
test_dataset = dataset_mt_te
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset (num_proc=4):   0%|          | 0/1965 [00:00<?, ? examples/s]

In [ ]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    load_best_model_at_end = True,
    evaluation_strategy = IntervalStrategy.STEPS,
    eval_steps = 500,
    metric_for_best_model = "eval_f1",
    num_train_epochs=4)

In [ ]:
# Initialize our Trainer
#early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
#args.metric_for_best_model = "f1"
#args.load_best_model_at_end = True
#args.evaluation_strategy = IntervalStrategy.STEPS
#args.eval_steps = args.save_steps
#args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #callbacks=[early_stopping_callback],
    args=args,
)

In [ ]:
train_result = trainer.train()  #seq_length=128

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.408000,0.242357,0.712518,0.555619,0.624362,0.940668
1000,0.198900,0.214808,0.657956,0.643019,0.650402,0.944377
1500,0.160400,0.201426,0.714003,0.613507,0.659951,0.947171
2000,0.133700,0.202965,0.643349,0.688990,0.665388,0.944072
2500,0.111000,0.199711,0.658242,0.679909,0.668900,0.945494
3000,0.085300,0.208790,0.660316,0.688422,0.674076,0.946155


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packa

KeyboardInterrupt: ignored

In [ ]:
train_result = trainer.train()    #lang id token maithili

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.407300,0.254181,0.720247,0.528644,0.609748,0.939345
1000,0.200900,0.207727,0.677262,0.628474,0.651956,0.945431
1500,0.160500,0.198704,0.723087,0.605786,0.659259,0.947307
2000,0.133300,0.199275,0.659934,0.685763,0.672601,0.946648
2500,0.107500,0.197581,0.668345,0.677822,0.673050,0.946800
3000,0.085200,0.205238,0.672222,0.686330,0.679203,0.947206


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packa

['<mai>',
 '<mai>',
 'नेताजीक',
 'संग',
 'फिल्मी',
 'कलाकार',
 'सेहो',
 'आबि',
 'रहल',
 'छथिन्ह',
 '।']

In [ ]:
enc=tokenizer.encode_plus('<mai> नेताजीक')

In [ ]:
enc['input_ids']

[1, 16, 114319, 9104, 2]

In [ ]:
mt_X_tr[0]

['<mai>',
 'नेताजीक',
 'संग',
 'फिल्मी',
 'कलाकार',
 'सेहो',
 'आबि',
 'रहल',
 'छथिन्ह',
 '।']

In [ ]:
train_result = trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.439900,0.268440,0.699634,0.541690,0.610614,0.936246
1000,0.216800,0.226788,0.662120,0.605786,0.632701,0.939213
1500,0.174300,0.213330,0.714665,0.610891,0.658716,0.943096
2000,0.148600,0.208877,0.665184,0.678389,0.671721,0.942665
2500,0.119700,0.204822,0.677364,0.670448,0.673888,0.943528
3000,0.096400,0.210963,0.673767,0.681792,0.677756,0.943474


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packa

In [ ]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9435
  eval_f1                 =     0.6778
  eval_loss               =      0.211
  eval_precision          =     0.6738
  eval_recall             =     0.6818
  eval_runtime            = 0:00:42.96
  eval_samples_per_second =     27.415
  eval_steps_per_second   =      3.444


In [ ]:
preds,labels,metrics2=trainer.predict(test_dataset,metric_key_prefix="predict")

In [ ]:
metrics2  #IndicBertV2 (Maithili only)

{'predict_loss': 0.19827055931091309,
 'predict_precision': 0.6767095463777928,
 'predict_recall': 0.6859986273164036,
 'predict_f1': 0.6813224267211997,
 'predict_accuracy': 0.9482333093817363,
 'predict_runtime': 71.0798,
 'predict_samples_per_second': 27.645,
 'predict_steps_per_second': 3.461}

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/mt_indicbertv2mlm')
model.save_pretrained('/content/drive/MyDrive/mt_indicbertv2mlm')

In [ ]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    metric_for_best_model ='eval_loss',
    load_best_model_at_end=True,
    evaluation_strategy = IntervalStrategy.STEPS, # "steps"
    eval_steps = 500,
    save_total_limit = 3,
    )

In [ ]:
# Initialize our Trainer
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=4)
#args.metric_for_best_model = "f1"
#args.load_best_model_at_end = True
#args.evaluation_strategy = IntervalStrategy.STEPS
#args.eval_steps = args.save_steps
#args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
    args=args,
)

In [ ]:
train_result = trainer.train()

Step,Training Loss,Validation Loss,Artifact Precision,Artifact Recall,Artifact F1,Artifact Number,Count Precision,Count Recall,Count F1,Count Number,Date Precision,Date Recall,Date F1,Date Number,Day Precision,Day Recall,Day F1,Day Number,Disease Precision,Disease Recall,Disease F1,Disease Number,Distance Precision,Distance Recall,Distance F1,Distance Number,Entertainment Precision,Entertainment Recall,Entertainment F1,Entertainment Number,Facility Precision,Facility Recall,Facility F1,Facility Number,Location Precision,Location Recall,Location F1,Location Number,Locomotive Precision,Locomotive Recall,Locomotive F1,Locomotive Number,Material Precision,Material Recall,Material F1,Material Number,Money Precision,Money Recall,Money F1,Money Number,Month Precision,Month Recall,Month F1,Month Number,Organism Precision,Organism Recall,Organism F1,Organism Number,Organization Precision,Organization Recall,Organization F1,Organization Number,Period Precision,Period Recall,Period F1,Period Number,Person Precision,Person Recall,Person F1,Person Number,Plant Precision,Plant Recall,Plant F1,Plant Number,Quantity Precision,Quantity Recall,Quantity F1,Quantity Number,Special Day Precision,Special Day Recall,Special Day F1,Special Day Number,Time Precision,Time Recall,Time F1,Time Number,Year Precision,Year Recall,Year F1,Year Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
500,0.057500,0.299094,0.456140,0.236364,0.311377,220,0.518987,0.518987,0.518987,316,0.000000,0.000000,0.000000,19,0.400000,0.125000,0.190476,32,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,2,0.292683,0.358209,0.322148,67,0.316832,0.533333,0.397516,60,0.576471,0.693069,0.629416,707,0.312500,0.571429,0.404040,35,0.404372,0.643478,0.496644,115,0.196721,0.352941,0.252632,34,0.521739,0.685714,0.592593,35,0.427184,0.282051,0.339768,156,0.492188,0.572727,0.529412,110,0.312500,0.408163,0.353982,49,0.614815,0.598846,0.606725,1386,0.296296,0.228571,0.258065,35,0.190476,0.062500,0.094118,64,0.380952,0.421053,0.400000,19,0.440678,0.247619,0.317073,105,0.500000,0.777778,0.608696,27,0.526749,0.533927,0.530313,0.935131
1000,0.223700,0.198718,0.529801,0.363636,0.431267,220,0.530488,0.550633,0.540373,316,0.000000,0.000000,0.000000,19,0.466667,0.218750,0.297872,32,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,2,0.481481,0.194030,0.276596,67,0.400000,0.300000,0.342857,60,0.789946,0.622348,0.696203,707,0.391304,0.514286,0.444444,35,0.666667,0.313043,0.426036,115,0.600000,0.088235,0.153846,34,0.742857,0.742857,0.742857,35,0.373737,0.474359,0.418079,156,0.581633,0.518182,0.548077,110,0.400000,0.326531,0.359551,49,0.739372,0.577201,0.648298,1386,0.666667,0.114286,0.195122,35,0.117647,0.062500,0.081633,64,0.416667,0.263158,0.322581,19,0.491228,0.266667,0.345679,105,0.809524,0.629630,0.708333,27,0.647226,0.506118,0.568040,0.945184
1500,0.183600,0.183783,0.653333,0.222727,0.332203,220,0.653333,0.465190,0.543438,316,0.000000,0.000000,0.000000,19,0.692308,0.281250,0.400000,32,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,2,0.583333,0.208955,0.307692,67,0.437500,0.350000,0.388889,60,0.865497,0.628006,0.727869,707,0.432432,0.457143,0.444444,35,0.479290,0.704348,0.570423,115,0.000000,0.000000,0.000000,34,0.823529,0.800000,0.811594,35,0.428571,0.019231,0.036810,156,0.432432,0.727273,0.542373,110,0.606061,0.408163,0.487805,49,0.769868,0.670996,0.717039,1386,0.636364,0.200000,0.304348,35,0.500000,0.015625,0.030303,64,0.600000,0.315789,0.413793,19,0.639344,0.371429,0.469880,105,1.000000,0.592593,0.744186,27,0.715195,0.531424,0.609764,0.949467
2000,0.158300,0.158347,0.540404,0.486364,0.511962,220,0.575221,0.617089,0.595420,316,0.714286,0.526316,0.606061,19,0.652174,0.468750,0.545455,32,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,2,0.512821,0.298507,0.377358,67,0.457143,0.533333,0.492308,60,0.804012,0.736917,0.769004,707,0.406780,0.685714,0.510638,35,0.565217,0.678261,0.616601,115,0.277778,0.294118,0.285714,34,0.837838,0.885714,0.861111,35,0.7407

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packa

In [ ]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)  #indicbertv2-ss combined dataset

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** eval metrics *****
  epoch                        =       2.79
  eval_Artifact_f1             =     0.5013
  eval_Artifact_number         =        220
  eval_Artifact_precision      =     0.5657
  eval_Artifact_recall         =       0.45
  eval_Count_f1                =     0.6012
  eval_Count_number            =        316
  eval_Count_precision         =      0.592
  eval_Count_recall            =     0.6108
  eval_Date_f1                 =     0.4828
  eval_Date_number             =         19
  eval_Date_precision          =        0.7
  eval_Date_recall             =     0.3684
  eval_Day_f1                  =     0.5417
  eval_Day_number              =         32
  eval_Day_precision           =     0.8125
  eval_Day_recall              =     0.4062
  eval_Disease_f1              =        0.0
  eval_Disease_number          =          3
  eval_Disease_precision       =        0.0
  eval_Disease_recall          =        0.0
  eval_Distance_f1             =        0.0
  eval_

In [ ]:
test_dataset_bh = dataset_bh_te
test_dataset_bh = test_dataset_bh.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset (num_proc=4):   0%|          | 0/4947 [00:00<?, ? examples/s]

In [ ]:
preds,labels,metrics2=trainer.predict(test_dataset_bh,metric_key_prefix="predict")

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


Combined model(IndicBertV2-SS) results on test set of bhojpuri language only

In [ ]:
metrics2

{'predict_loss': 0.09075689315795898,
 'predict_Artifact_precision': 0.552,
 'predict_Artifact_recall': 0.40588235294117647,
 'predict_Artifact_f1': 0.46779661016949153,
 'predict_Artifact_number': 170,
 'predict_Count_precision': 0.3939393939393939,
 'predict_Count_recall': 0.3270440251572327,
 'predict_Count_f1': 0.35738831615120276,
 'predict_Count_number': 159,
 'predict_Date_precision': 0.375,
 'predict_Date_recall': 0.35294117647058826,
 'predict_Date_f1': 0.3636363636363636,
 'predict_Date_number': 17,
 'predict_Day_precision': 0.42857142857142855,
 'predict_Day_recall': 0.3333333333333333,
 'predict_Day_f1': 0.375,
 'predict_Day_number': 9,
 'predict_Disease_precision': 0.3333333333333333,
 'predict_Disease_recall': 0.1111111111111111,
 'predict_Disease_f1': 0.16666666666666666,
 'predict_Disease_number': 9,
 'predict_Distance_precision': 0.5,
 'predict_Distance_recall': 0.3333333333333333,
 'predict_Distance_f1': 0.4,
 'predict_Distance_number': 3,
 'predict_Entertainment_prec

In [ ]:
test_dataset_bh = dataset_bh_te
test_dataset_bh = dataset_bh_te.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset (num_proc=4):   0%|          | 0/4947 [00:00<?, ? examples/s]

In [ ]:
train_result = trainer.train('/content/output_dir/checkpoint-3500')

Step,Training Loss,Validation Loss,Artifact Precision,Artifact Recall,Artifact F1,Artifact Number,Count Precision,Count Recall,Count F1,Count Number,Date Precision,Date Recall,Date F1,Date Number,Day Precision,Day Recall,Day F1,Day Number,Disease Precision,Disease Recall,Disease F1,Disease Number,Distance Precision,Distance Recall,Distance F1,Distance Number,Entertainment Precision,Entertainment Recall,Entertainment F1,Entertainment Number,Facility Precision,Facility Recall,Facility F1,Facility Number,Location Precision,Location Recall,Location F1,Location Number,Locomotive Precision,Locomotive Recall,Locomotive F1,Locomotive Number,Material Precision,Material Recall,Material F1,Material Number,Money Precision,Money Recall,Money F1,Money Number,Month Precision,Month Recall,Month F1,Month Number,Organism Precision,Organism Recall,Organism F1,Organism Number,Organization Precision,Organization Recall,Organization F1,Organization Number,Period Precision,Period Recall,Period F1,Period Number,Person Precision,Person Recall,Person F1,Person Number,Plant Precision,Plant Recall,Plant F1,Plant Number,Quantity Precision,Quantity Recall,Quantity F1,Quantity Number,Special Day Precision,Special Day Recall,Special Day F1,Special Day Number,Time Precision,Time Recall,Time F1,Time Number,Year Precision,Year Recall,Year F1,Year Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
4000,0.158900,0.199182,0.508475,0.191083,0.277778,157,0.546729,0.596939,0.570732,196,0.500000,0.300000,0.375000,10,0.312500,0.294118,0.303030,17,0.000000,0.000000,0.000000,6,0.000000,0.000000,0.000000,2,0.323529,0.159420,0.213592,69,0.488372,0.477273,0.482759,44,0.729282,0.707775,0.718367,373,0.428571,0.150000,0.222222,20,0.450980,0.383333,0.414414,60,0.294118,0.250000,0.270270,20,0.666667,0.785714,0.721311,28,0.428571,0.183673,0.257143,98,0.573034,0.531250,0.551351,96,0.550000,0.333333,0.415094,33,0.701597,0.744703,0.722508,944,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,28,0.000000,0.000000,0.000000,7,0.678571,0.322034,0.436782,59,0.666667,0.583333,0.622222,24,0.643588,0.573913,0.606757,0.948976
4500,0.150500,0.192476,0.394495,0.273885,0.323308,157,0.592391,0.556122,0.573684,196,0.545455,0.600000,0.571429,10,0.666667,0.117647,0.200000,17,0.000000,0.000000,0.000000,6,0.000000,0.000000,0.000000,2,0.277778,0.144928,0.190476,69,0.578947,0.500000,0.536585,44,0.773810,0.697051,0.733427,373,0.454545,0.250000,0.322581,20,0.410256,0.266667,0.323232,60,0.142857,0.250000,0.181818,20,0.750000,0.857143,0.800000,28,0.415584,0.326531,0.365714,98,0.678571,0.593750,0.633333,96,0.325581,0.424242,0.368421,33,0.771715,0.734110,0.752443,944,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,28,0.125000,0.142857,0.133333,7,0.594595,0.372881,0.458333,59,0.923077,0.500000,0.648649,24,0.668170,0.579565,0.620722,0.951045
5000,0.149500,0.196186,0.394495,0.273885,0.323308,157,0.531532,0.602041,0.564593,196,0.600000,0.300000,0.400000,10,0.375000,0.176471,0.240000,17,0.000000,0.000000,0.000000,6,0.000000,0.000000,0.000000,2,0.448276,0.188406,0.265306,69,0.485714,0.386364,0.430380,44,0.654676,0.731903,0.691139,373,0.444444,0.200000,0.275862,20,0.370370,0.333333,0.350877,60,0.208333,0.250000,0.227273,20,0.709677,0.785714,0.745763,28,0.416667,0.204082,0.273973,98,0.695122,0.593750,0.640449,96,0.391304,0.272727,0.321429,33,0.749462,0.738347,0.743863,944,0.000000,0.000000,0.000000,9,0.041667,0.035714,0.038462,28,0.333333,0.142857,0.200000,7,0.555556,0.508475,0.530973,59,0.705882,0.500000,0.585366,24,0.634652,0.586087,0.609403,0.949903
5500,0.142200,0.201188,0.410000,0.261146,0.319066,157,0.539604,0.556122,0.547739,196,0.545455,0.600000,0.571429,10,0.428571,0.176471,0.250000,17,0.000000,0.000000,0.000000,6,0.000000,0.000000,0.000000,2,0.486486,0.260870,0.339623,69,0.547619,0.522727,0.534884,44,0.706037,0.721180,0.713528,373,0.444444,0.200000,0.275862,20,0.411765,0.350000,0.378378,60,0.333333,0.300000,0.315789,20,0.793103,0.821429,0.807018,28,0.519231,0.275510,0.360000,

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packa

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/bmm_model_com')
model.save_pretrained('/content/drive/MyDrive/bmm_model_com')

In [ ]:
!zip -r /content/bmm_modelc_check.zip /content/output_dir/checkpoint-6500

  adding: content/output_dir/checkpoint-6500/ (stored 0%)
  adding: content/output_dir/checkpoint-6500/training_args.bin (deflated 51%)
  adding: content/output_dir/checkpoint-6500/tokenizer.json (deflated 69%)
  adding: content/output_dir/checkpoint-6500/special_tokens_map.json (deflated 42%)
  adding: content/output_dir/checkpoint-6500/config.json (deflated 68%)
  adding: content/output_dir/checkpoint-6500/trainer_state.json (deflated 88%)
  adding: content/output_dir/checkpoint-6500/model.safetensors (deflated 7%)
  adding: content/output_dir/checkpoint-6500/optimizer.pt (deflated 52%)
  adding: content/output_dir/checkpoint-6500/vocab.txt (deflated 48%)
  adding: content/output_dir/checkpoint-6500/scheduler.pt (deflated 56%)
  adding: content/output_dir/checkpoint-6500/tokenizer_config.json (deflated 75%)
  adding: content/output_dir/checkpoint-6500/rng_state.pth (deflated 25%)


In [ ]:
train_result = trainer.train('/content/output_dir/checkpoint-7000')

Step,Training Loss,Validation Loss,Artifact Precision,Artifact Recall,Artifact F1,Artifact Number,Count Precision,Count Recall,Count F1,Count Number,Date Precision,Date Recall,Date F1,Date Number,Day Precision,Day Recall,Day F1,Day Number,Disease Precision,Disease Recall,Disease F1,Disease Number,Distance Precision,Distance Recall,Distance F1,Distance Number,Entertainment Precision,Entertainment Recall,Entertainment F1,Entertainment Number,Facility Precision,Facility Recall,Facility F1,Facility Number,Location Precision,Location Recall,Location F1,Location Number,Locomotive Precision,Locomotive Recall,Locomotive F1,Locomotive Number,Material Precision,Material Recall,Material F1,Material Number,Money Precision,Money Recall,Money F1,Money Number,Month Precision,Month Recall,Month F1,Month Number,Organism Precision,Organism Recall,Organism F1,Organism Number,Organization Precision,Organization Recall,Organization F1,Organization Number,Period Precision,Period Recall,Period F1,Period Number,Person Precision,Person Recall,Person F1,Person Number,Plant Precision,Plant Recall,Plant F1,Plant Number,Quantity Precision,Quantity Recall,Quantity F1,Quantity Number,Special Day Precision,Special Day Recall,Special Day F1,Special Day Number,Time Precision,Time Recall,Time F1,Time Number,Year Precision,Year Recall,Year F1,Year Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
7500,0.108900,0.197660,0.385965,0.280255,0.324723,157,0.591837,0.591837,0.591837,196,0.583333,0.700000,0.636364,10,0.500000,0.176471,0.260870,17,0.000000,0.000000,0.000000,6,0.000000,0.000000,0.000000,2,0.428571,0.217391,0.288462,69,0.523810,0.500000,0.511628,44,0.762178,0.713137,0.736842,373,0.454545,0.250000,0.322581,20,0.466667,0.350000,0.400000,60,0.333333,0.400000,0.363636,20,0.800000,0.857143,0.827586,28,0.577778,0.265306,0.363636,98,0.694118,0.614583,0.651934,96,0.406250,0.393939,0.400000,33,0.780973,0.747881,0.764069,944,0.000000,0.000000,0.000000,9,0.130435,0.107143,0.117647,28,0.000000,0.000000,0.000000,7,0.608696,0.474576,0.533333,59,0.857143,0.500000,0.631579,24,0.682516,0.599130,0.638111,0.952775


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
preds_bh,labels_bh,metrics_bh=trainer.predict(test_dataset_bh,metric_key_prefix="predict_bh")

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Combined model(IndicNER) results on test set of bhojpuri language only**

In [ ]:
metrics_bh

{'predict_bh_loss': 0.12265395373106003,
 'predict_bh_Artifact_precision': 0.2787878787878788,
 'predict_bh_Artifact_recall': 0.27058823529411763,
 'predict_bh_Artifact_f1': 0.27462686567164185,
 'predict_bh_Artifact_number': 170,
 'predict_bh_Count_precision': 0.37575757575757573,
 'predict_bh_Count_recall': 0.389937106918239,
 'predict_bh_Count_f1': 0.38271604938271603,
 'predict_bh_Count_number': 159,
 'predict_bh_Date_precision': 0.4375,
 'predict_bh_Date_recall': 0.8235294117647058,
 'predict_bh_Date_f1': 0.5714285714285714,
 'predict_bh_Date_number': 17,
 'predict_bh_Day_precision': 0.25,
 'predict_bh_Day_recall': 0.1111111111111111,
 'predict_bh_Day_f1': 0.15384615384615383,
 'predict_bh_Day_number': 9,
 'predict_bh_Disease_precision': 0.0,
 'predict_bh_Disease_recall': 0.0,
 'predict_bh_Disease_f1': 0.0,
 'predict_bh_Disease_number': 9,
 'predict_bh_Distance_precision': 0.0,
 'predict_bh_Distance_recall': 0.0,
 'predict_bh_Distance_f1': 0.0,
 'predict_bh_Distance_number': 3,
 '

**Model using only Maithili language(45 labels)-Indic NER**

In [ ]:
train_dataset_mt = dataset_mt_ev['train']
train_dataset_mt = train_dataset_mt.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset (num_proc=4):   0%|          | 0/7063 [00:00<?, ? examples/s]

In [ ]:
eval_dataset_mt = dataset_mt_ev['test']
eval_dataset_mt = eval_dataset_mt.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on validation dataset",
)

Running tokenizer on validation dataset (num_proc=4):   0%|          | 0/785 [00:00<?, ? examples/s]

In [ ]:
test_dataset_mt = dataset_mt_te
test_dataset_mt = test_dataset_mt.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset (num_proc=4):   0%|          | 0/1965 [00:00<?, ? examples/s]

In [ ]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    metric_for_best_model ='eval_loss',
    load_best_model_at_end=True,
    evaluation_strategy = IntervalStrategy.STEPS, # "steps"
    eval_steps = 500,
    save_total_limit = 5)

In [ ]:
# Initialize our Trainer
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=4)
#args.metric_for_best_model = "f1"
#args.load_best_model_at_end = True
#args.evaluation_strategy = IntervalStrategy.STEPS
#args.eval_steps = args.save_steps
#args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset_mt,
    eval_dataset=eval_dataset_mt,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
    args=args,
)

In [ ]:
train_result = trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Artifact Precision,Artifact Recall,Artifact F1,Artifact Number,Count Precision,Count Recall,Count F1,Count Number,Date Precision,Date Recall,Date F1,Date Number,Day Precision,Day Recall,Day F1,Day Number,Disease Precision,Disease Recall,Disease F1,Disease Number,Distance Precision,Distance Recall,Distance F1,Distance Number,Entertainment Precision,Entertainment Recall,Entertainment F1,Entertainment Number,Facility Precision,Facility Recall,Facility F1,Facility Number,Location Precision,Location Recall,Location F1,Location Number,Locomotive Precision,Locomotive Recall,Locomotive F1,Locomotive Number,Material Precision,Material Recall,Material F1,Material Number,Money Precision,Money Recall,Money F1,Money Number,Month Precision,Month Recall,Month F1,Month Number,Organism Precision,Organism Recall,Organism F1,Organism Number,Organization Precision,Organization Recall,Organization F1,Organization Number,Period Precision,Period Recall,Period F1,Period Number,Person Precision,Person Recall,Person F1,Person Number,Quantity Precision,Quantity Recall,Quantity F1,Quantity Number,Special Day Precision,Special Day Recall,Special Day F1,Special Day Number,Time Precision,Time Recall,Time F1,Time Number,Year Precision,Year Recall,Year F1,Year Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
500,0.644400,0.393606,0.000000,0.000000,0.000000,48,0.518519,0.101449,0.169697,138,0.000000,0.000000,0.000000,2,0.000000,0.000000,0.000000,5,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,29,0.000000,0.000000,0.000000,33,0.587859,0.619529,0.603279,297,0.000000,0.000000,0.000000,13,0.000000,0.000000,0.000000,44,0.000000,0.000000,0.000000,14,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,12,0.400000,0.439560,0.418848,91,0.000000,0.000000,0.000000,21,0.537217,0.535484,0.536349,310,0.000000,0.000000,0.000000,7,0.000000,0.000000,0.000000,13,0.000000,0.000000,0.000000,19,0.000000,0.000000,0.000000,13,0.539386,0.355634,0.428647,0.918925
1000,0.340900,0.304611,0.000000,0.000000,0.000000,48,0.516304,0.688406,0.590062,138,0.000000,0.000000,0.000000,2,0.000000,0.000000,0.000000,5,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,29,0.266667,0.121212,0.166667,33,0.669811,0.717172,0.692683,297,0.000000,0.000000,0.000000,13,0.000000,0.000000,0.000000,44,0.000000,0.000000,0.000000,14,0.904762,0.760000,0.826087,25,0.000000,0.000000,0.000000,12,0.471154,0.538462,0.502564,91,0.000000,0.000000,0.000000,21,0.577049,0.567742,0.572358,310,0.000000,0.000000,0.000000,7,0.000000,0.000000,0.000000,13,0.000000,0.000000,0.000000,19,1.000000,0.846154,0.916667,13,0.591241,0.499120,0.541289,0.931706
1500,0.266600,0.282057,0.105263,0.041667,0.059701,48,0.657143,0.666667,0.661871,138,0.000000,0.000000,0.000000,2,0.000000,0.000000,0.000000,5,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,1,0.500000,0.068966,0.121212,29,0.500000,0.363636,0.421053,33,0.687097,0.717172,0.701812,297,0.000000,0.000000,0.000000,13,0.444444,0.090909,0.150943,44,0.000000,0.000000,0.000000,14,0.884615,0.920000,0.901961,25,0.000000,0.000000,0.000000,12,0.593407,0.593407,0.593407,91,0.263158,0.238095,0.250000,21,0.590323,0.590323,0.590323,310,0.000000,0.000000,0.000000,7,0.000000,0.000000,0.000000,13,0.000000,0.000000,0.000000,19,0.916667,0.846154,0.880000,13,0.623444,0.529049,0.572381,0.936557
2000,0.218200,0.277921,0.266667,0.083333,0.126984,48,0.676056,0.695652,0.685714,138,0.000000,0.000000,0.000000,2,0.000000,0.000000,0.000000,5,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,1,0.244444,0.379310,0.297297,29,0.407407,0.333333,0.366667,33,0.713826,0.747475,0.730263,297,0.000000,0.000000,0.000000,13,0.387097,0.272727,0.320000,44,0.000000,0.000000,0.000000,14,0.920000,0.920000,0.920000,25,0.000000,0.000000,0.000000,12,0.602151,0.615385,0.608696,91,0.185185,0.238095,0.208333,21,0.574468,0.609677,0.591549,310,0.000000,0.000000,0.000000,7,0.500000,0.153846,0.235294,1

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packa

In [ ]:
metrics = train_result.metrics

In [ ]:
metrics

{'train_runtime': 2204.6447,
 'train_samples_per_second': 9.611,
 'train_steps_per_second': 1.202,
 'total_flos': 5538769395747840.0,
 'train_loss': 0.3238192205835892,
 'epoch': 3.0}

In [ ]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** eval metrics *****
  epoch                        =        3.0
  eval_Artifact_f1             =     0.1978
  eval_Artifact_number         =         48
  eval_Artifact_precision      =     0.2093
  eval_Artifact_recall         =     0.1875
  eval_Count_f1                =     0.7046
  eval_Count_number            =        138
  eval_Count_precision         =     0.6923
  eval_Count_recall            =     0.7174
  eval_Date_f1                 =        0.5
  eval_Date_number             =          2
  eval_Date_precision          =        0.5
  eval_Date_recall             =        0.5
  eval_Day_f1                  =        0.0
  eval_Day_number              =          5
  eval_Day_precision           =        0.0
  eval_Day_recall              =        0.0
  eval_Disease_f1              =        0.0
  eval_Disease_number          =          1
  eval_Disease_precision       =        0.0
  eval_Disease_recall          =        0.0
  eval_Distance_f1             =        0.0
  eval_

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/bmm_model_mt')
model.save_pretrained('/content/drive/MyDrive/bmm_model_mt')

In [ ]:
preds_mt,labels_mt,metrics_mt=trainer.predict(test_dataset_mt,metric_key_prefix="predict_mt")

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
metrics_mt

{'predict_mt_loss': 0.2670222520828247,
 'predict_mt_Artifact_precision': 0.1927710843373494,
 'predict_mt_Artifact_recall': 0.14678899082568808,
 'predict_mt_Artifact_f1': 0.16666666666666669,
 'predict_mt_Artifact_number': 109,
 'predict_mt_Count_precision': 0.5211267605633803,
 'predict_mt_Count_recall': 0.6045751633986928,
 'predict_mt_Count_f1': 0.5597579425113465,
 'predict_mt_Count_number': 306,
 'predict_mt_Date_precision': 0.8571428571428571,
 'predict_mt_Date_recall': 0.6,
 'predict_mt_Date_f1': 0.7058823529411764,
 'predict_mt_Date_number': 10,
 'predict_mt_Day_precision': 0.0,
 'predict_mt_Day_recall': 0.0,
 'predict_mt_Day_f1': 0.0,
 'predict_mt_Day_number': 25,
 'predict_mt_Disease_precision': 0.0,
 'predict_mt_Disease_recall': 0.0,
 'predict_mt_Disease_f1': 0.0,
 'predict_mt_Disease_number': 4,
 'predict_mt_Distance_precision': 0.0,
 'predict_mt_Distance_recall': 0.0,
 'predict_mt_Distance_f1': 0.0,
 'predict_mt_Distance_number': 4,
 'predict_mt_Entertainment_precision':

**Model using only Magahi language(All 45 labels)-Indic NER**

In [ ]:
train_dataset_mg = dataset_mg_ev['train']
train_dataset_mg = train_dataset_mg.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset (num_proc=4):   0%|          | 0/3828 [00:00<?, ? examples/s]

In [ ]:
eval_dataset_mg = dataset_mg_ev['test']
eval_dataset_mg = eval_dataset_mg.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on validation dataset",
)

Running tokenizer on validation dataset (num_proc=4):   0%|          | 0/426 [00:00<?, ? examples/s]

In [ ]:
test_dataset_mg = dataset_mg_te
test_dataset_mg = test_dataset_mg.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset (num_proc=4):   0%|          | 0/1064 [00:00<?, ? examples/s]

In [ ]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
args1=TrainingArguments(
    output_dir='output_dir1',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    metric_for_best_model ='eval_loss',
    load_best_model_at_end=True,
    evaluation_strategy = IntervalStrategy.STEPS, # "steps"
    eval_steps = 250,
    save_total_limit = 5)

In [ ]:
# Initialize our Trainer
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=4)
#args.metric_for_best_model = "f1"
#args.load_best_model_at_end = True
#args.evaluation_strategy = IntervalStrategy.STEPS
#args.eval_steps = args.save_steps
#args.greater_is_better = True

trainer1 = Trainer(
    model=model,
    train_dataset=train_dataset_mg,
    eval_dataset=eval_dataset_mg,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
    args=args1,
)

In [ ]:
train_result1 = trainer1.train()

Step,Training Loss,Validation Loss,Artifact Precision,Artifact Recall,Artifact F1,Artifact Number,Count Precision,Count Recall,Count F1,Count Number,Day Precision,Day Recall,Day F1,Day Number,Disease Precision,Disease Recall,Disease F1,Disease Number,Entertainment Precision,Entertainment Recall,Entertainment F1,Entertainment Number,Facility Precision,Facility Recall,Facility F1,Facility Number,Location Precision,Location Recall,Location F1,Location Number,Locomotive Precision,Locomotive Recall,Locomotive F1,Locomotive Number,Material Precision,Material Recall,Material F1,Material Number,Money Precision,Money Recall,Money F1,Money Number,Month Precision,Month Recall,Month F1,Month Number,Organism Precision,Organism Recall,Organism F1,Organism Number,Organization Precision,Organization Recall,Organization F1,Organization Number,Period Precision,Period Recall,Period F1,Period Number,Person Precision,Person Recall,Person F1,Person Number,Plant Precision,Plant Recall,Plant F1,Plant Number,Quantity Precision,Quantity Recall,Quantity F1,Quantity Number,Time Precision,Time Recall,Time F1,Time Number,Year Precision,Year Recall,Year F1,Year Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
250,No log,0.308371,1.000000,0.160000,0.275862,50,0.645833,0.673913,0.659574,46,0.545455,0.857143,0.666667,21,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,7,0.540000,0.600000,0.568421,45,0.000000,0.000000,0.000000,4,1.000000,0.083333,0.153846,24,0.000000,0.000000,0.000000,8,0.500000,0.444444,0.470588,9,0.461538,0.146341,0.222222,41,0.000000,0.000000,0.000000,2,0.000000,0.000000,0.000000,3,0.752475,0.700461,0.725537,217,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,4,0.750000,0.111111,0.193548,27,0.400000,0.666667,0.500000,3,0.678284,0.492218,0.570462,0.918171
500,0.349500,0.260700,0.428571,0.360000,0.391304,50,0.642857,0.782609,0.705882,46,0.720000,0.857143,0.782609,21,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,1,1.000000,0.857143,0.923077,7,0.562500,0.800000,0.660550,45,0.000000,0.000000,0.000000,4,0.437500,0.291667,0.350000,24,0.000000,0.000000,0.000000,8,1.000000,0.444444,0.615385,9,0.531915,0.609756,0.568182,41,0.000000,0.000000,0.000000,2,0.000000,0.000000,0.000000,3,0.771429,0.746544,0.758782,217,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,4,0.666667,0.814815,0.733333,27,0.400000,0.666667,0.500000,3,0.660118,0.653696,0.656891,0.929660
750,0.349500,0.249146,0.527778,0.380000,0.441860,50,0.685185,0.804348,0.740000,46,0.894737,0.809524,0.850000,21,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,1,1.000000,0.857143,0.923077,7,0.631579,0.800000,0.705882,45,1.000000,0.250000,0.400000,4,0.400000,0.333333,0.363636,24,0.750000,0.375000,0.500000,8,0.636364,0.777778,0.700000,9,0.545455,0.585366,0.564706,41,0.000000,0.000000,0.000000,2,0.000000,0.000000,0.000000,3,0.670455,0.815668,0.735967,217,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,4,0.769231,0.740741,0.754717,27,1.000000,0.666667,0.800000,3,0.655046,0.694553,0.674221,0.930129
1000,0.195600,0.254229,0.486486,0.360000,0.413793,50,0.661017,0.847826,0.742857,46,0.692308,0.857143,0.765957,21,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,1,1.000000,0.857143,0.923077,7,0.629032,0.866667,0.728972,45,1.000000,0.250000,0.400000,4,0.391304,0.375000,0.382979,24,0.500000,0.500000,0.500000,8,1.000000,0.777778,0.875000,9,0.600000,0.658537,0.627907,41,0.000000,0.000000,0.000000,2,0.000000,0.000000,0.000000,3,0.713725,0.838710,0.771186,217,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,4,0.636364,0.777778,0.700000,27,1.000000,0.666667,0.800000,3,0.661348,0.725681,0.692022,0.934115
1250,0.195600,0.246510,0.507937,0.640000,0.566372,50,0.694915,0.891304,0.780952,46,0.720000,0.857143,0.782609,21,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,1,1.000000,0.857143,0.923077,7,0.614035,0.777778,0.686275,45,1.000000,0.250000,0.400000,4,0.380952,0.333333,0.355556,24,0.363636,0.500000,0.421

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packa

In [ ]:
metrics1 = trainer1.evaluate()

***** eval metrics *****
  epoch                        =        3.0
  eval_Artifact_f1             =     0.1978
  eval_Artifact_number         =         48
  eval_Artifact_precision      =     0.2093
  eval_Artifact_recall         =     0.1875
  eval_Count_f1                =     0.7046
  eval_Count_number            =        138
  eval_Count_precision         =     0.6923
  eval_Count_recall            =     0.7174
  eval_Date_f1                 =        0.5
  eval_Date_number             =          2
  eval_Date_precision          =        0.5
  eval_Date_recall             =        0.5
  eval_Day_f1                  =        0.0
  eval_Day_number              =          5
  eval_Day_precision           =        0.0
  eval_Day_recall              =        0.0
  eval_Disease_f1              =        0.0
  eval_Disease_number          =          1
  eval_Disease_precision       =        0.0
  eval_Disease_recall          =        0.0
  eval_Distance_f1             =        0.0
  eval_

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
trainer1.log_metrics("eval", metrics1)

***** eval metrics *****
  epoch                        =        3.0
  eval_Artifact_f1             =     0.4138
  eval_Artifact_number         =         50
  eval_Artifact_precision      =     0.4865
  eval_Artifact_recall         =       0.36
  eval_Count_f1                =     0.7429
  eval_Count_number            =         46
  eval_Count_precision         =      0.661
  eval_Count_recall            =     0.8478
  eval_Day_f1                  =      0.766
  eval_Day_number              =         21
  eval_Day_precision           =     0.6923
  eval_Day_recall              =     0.8571
  eval_Disease_f1              =        0.0
  eval_Disease_number          =          1
  eval_Disease_precision       =        0.0
  eval_Disease_recall          =        0.0
  eval_Entertainment_f1        =        0.0
  eval_Entertainment_number    =          1
  eval_Entertainment_precision =        0.0
  eval_Entertainment_recall    =        0.0
  eval_Facility_f1             =     0.9231
  eval_

In [ ]:
len(eval_dataset_mg)

426

In [ ]:
len(eval_dataset_mt)

785

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/bmm_model_mg')
model.save_pretrained('/content/drive/MyDrive/bmm_model_mg')

In [ ]:
preds_mg,labels_mg,metrics_mg=trainer1.predict(test_dataset_mg,metric_key_prefix="predict_mg")

In [ ]:
metrics_mg

{'predict_mg_loss': 0.2614980936050415,
 'predict_mg_Artifact_precision': 0.5891472868217055,
 'predict_mg_Artifact_recall': 0.5,
 'predict_mg_Artifact_f1': 0.5409252669039145,
 'predict_mg_Artifact_number': 152,
 'predict_mg_Count_precision': 0.668918918918919,
 'predict_mg_Count_recall': 0.7795275590551181,
 'predict_mg_Count_f1': 0.7200000000000001,
 'predict_mg_Count_number': 127,
 'predict_mg_Date_precision': 0.0,
 'predict_mg_Date_recall': 0.0,
 'predict_mg_Date_f1': 0.0,
 'predict_mg_Date_number': 3,
 'predict_mg_Day_precision': 0.5833333333333334,
 'predict_mg_Day_recall': 0.7241379310344828,
 'predict_mg_Day_f1': 0.6461538461538462,
 'predict_mg_Day_number': 29,
 'predict_mg_Disease_precision': 0.0,
 'predict_mg_Disease_recall': 0.0,
 'predict_mg_Disease_f1': 0.0,
 'predict_mg_Disease_number': 3,
 'predict_mg_Distance_precision': 0.0,
 'predict_mg_Distance_recall': 0.0,
 'predict_mg_Distance_f1': 0.0,
 'predict_mg_Distance_number': 1,
 'predict_mg_Entertainment_precision': 0.0